In [1]:
import os
import pandas as pd
from BaselineRemoval import BaselineRemoval
from scipy import signal
import pickle
import similarity as sim

def find_peaks(y, cwt=False, smooth=False, baseline=False):
    window, deg = 15, 2     
    z = y
    if smooth:
        z = signal.savgol_filter(y, window, deg, deriv=0)

    if baseline:
        brm = BaselineRemoval(z)
        z = brm.ZhangFit(lambda_=400,repitition=15, porder=1)
    if cwt == True:
        peaks = signal.find_peaks_cwt(z, [20])  
    else:
        dist, prom = 100, 600
        peaks,_ = signal.find_peaks(z, distance=dist, prominence=prom)
    return peaks, z

def similar_to(pk1, pk2, method='jaccard', rank=5):
    if method == 'rank':
        return sim.rank_similarity(pk1, pk2, good_with=rank)
    if method == 'weighted':
        return sim.rank_similarity(pk1, pk2, good_with=rank, weighted=True)
    else:
        return sim.jaccard_similarity(pk1, pk2)
    

DATA_DIR_PATH = "datasets/export"
PEAK_DIR_PATH = "peaks"
files =[
    "L185_0_A9_1", "L185_0_A10_1",
    "L186_0_A11_1", "L186_0_A12_1",
    "L187_0_B1_1", "L187_0_B2_1",
    "L291_0_A1_1", "L291_0_A2_1",
    "L100_0_G7_1", "L100_0_G8_1",
    "L101_0_A1_1", "L101_0_A2_1",
    "L103_0_A5_1", "L103_0_A6_1",
    "L125_0_E1_1", "L125_0_E2_1",
    "L126_0_E3_1", "L126_0_E4_1",
    "L128_0_E7_1", "L128_0_E8_1",   
]
peaks_extracted = []
n = len(files)
for i in range(n):
    df = None
    df = pd.read_table(f"{DATA_DIR_PATH}/{files[i]}.txt",sep=" ", header=None,names=['m/z', 'intensity']) 
    x, y = df['m/z'].to_numpy(), df['intensity'].to_numpy()

    #  Peak detection in new datasets (time-consuming) 

    #     %time peaks, _ =  find_peaks(y, cwt=True, smooth=False, baseline=False)
    #     print(f'{files[i]} peaks: {len(peaks)}')
    #     with  open(f'{PEAK_DIR_PATH}/{files[i]}_peaks.pkl', 'wb') as peak_file
    #         pickle.dump(peaks, peak_file)
    #         peaks_extracted += [(x[peaks], y[peaks])]

    #  Read peaks from pickle files

    with open(f'{PEAK_DIR_PATH}/{files[i]}_peaks.pkl', 'rb') as peak_file:
        peaks = pickle.load(peak_file)
        print(f'{files[i]} peaks: {len(peaks)}')
        peaks_extracted +=  [(x[peaks], y[peaks]) ] 


L185_0_A9_1 peaks: 195
L185_0_A10_1 peaks: 202
L186_0_A11_1 peaks: 213
L186_0_A12_1 peaks: 219
L187_0_B1_1 peaks: 211
L187_0_B2_1 peaks: 215
L291_0_A1_1 peaks: 215
L291_0_A2_1 peaks: 201
L100_0_G7_1 peaks: 207
L100_0_G8_1 peaks: 224
L101_0_A1_1 peaks: 229
L101_0_A2_1 peaks: 200
L103_0_A5_1 peaks: 195
L103_0_A6_1 peaks: 197
L125_0_E1_1 peaks: 209
L125_0_E2_1 peaks: 192
L126_0_E3_1 peaks: 197
L126_0_E4_1 peaks: 198
L128_0_E7_1 peaks: 204
L128_0_E8_1 peaks: 193


In [2]:
import importlib
n = len(files)

In [43]:
importlib.reload(sim)
rs = []
print('Rank_Similarity:')
for i in range(n-1):
    for j in range(i+1,n):
        score = similar_to(peaks_extracted[i], peaks_extracted[j], method='rank', rank=2)
        rs += [(i, j, score)]

rs = sorted(rs, key=lambda x: x[2], reverse=True)
algo_score=0
for k, v in enumerate(rs):
    i, j, s = v
    tag = '*' if files[i][:4]==files[j][:4] else '' # tag the same strain id
    if tag=='*':
        print(f'#{k+1:2d}{tag}:\t{files[i]} & {files[j]}\t{s:.4f}{tag}')
        algo_score += k;

p = n/2      # pairs of files   
print(f'algo_score={2*algo_score/float(p*(p+1)):.2f} (closer to 1.0 is better )')

Rank_Similarity:
# 1*:	L128_0_E7_1 & L128_0_E8_1	0.1397*
# 6*:	L100_0_G7_1 & L100_0_G8_1	0.0810*
# 7*:	L185_0_A9_1 & L185_0_A10_1	0.0791*
#11*:	L186_0_A11_1 & L186_0_A12_1	0.0696*
#26*:	L125_0_E1_1 & L125_0_E2_1	0.0588*
#29*:	L291_0_A1_1 & L291_0_A2_1	0.0581*
#36*:	L187_0_B1_1 & L187_0_B2_1	0.0538*
#54*:	L101_0_A1_1 & L101_0_A2_1	0.0467*
#55*:	L103_0_A5_1 & L103_0_A6_1	0.0464*
#64*:	L126_0_E3_1 & L126_0_E4_1	0.0440*
algo_score=5.07 (closer to 1.0 is better )


In [45]:
importlib.reload(sim)
rs = []
print('Weighted_Similarity:')
for i in range(n-1):
    for j in range(i+1,n):
        score = similar_to(peaks_extracted[i], peaks_extracted[j], method='weighted', rank=2)
        rs += [(i, j, score)]

rs = sorted(rs, key=lambda x: x[2], reverse=True)
algo_score=0
for k, v in enumerate(rs):
    i, j, s = v
    tag = '*' if files[i][:4]==files[j][:4] else '' # tag the same strain id
    if tag=='*':
        print(f'#{k+1:2d}{tag}:\t{files[i]} & {files[j]}\t{s:.4f}{tag}')
        algo_score += k;      

p = n/2      # pairs of files         
print(f'algo_score={2*algo_score/float(p*(p+1)):.2f} (closer to 1.0 is better )')

Weighted_Similarity:
# 1*:	L128_0_E7_1 & L128_0_E8_1	0.0970*
# 6*:	L185_0_A9_1 & L185_0_A10_1	0.0602*
# 8*:	L100_0_G7_1 & L100_0_G8_1	0.0576*
#18*:	L125_0_E1_1 & L125_0_E2_1	0.0499*
#22*:	L291_0_A1_1 & L291_0_A2_1	0.0480*
#23*:	L186_0_A11_1 & L186_0_A12_1	0.0470*
#27*:	L187_0_B1_1 & L187_0_B2_1	0.0459*
#37*:	L101_0_A1_1 & L101_0_A2_1	0.0439*
#50*:	L126_0_E3_1 & L126_0_E4_1	0.0413*
#62*:	L103_0_A5_1 & L103_0_A6_1	0.0364*
algo_score=4.44 (closer to 1.0 is better )
